In [57]:
receivable1 = ["Invoice"
              ,{"start":"2024-04-01","originBalance":2000
                 ,"originAdvance":1500,"dueDate":"2024-08-01"
                ,"feeType":("Fixed",150)}
              ,{"status":"Current"}]
receivable4 = ["Invoice"
              ,{"start":"2024-04-01","originBalance":2000
                 ,"originAdvance":1500,"dueDate":"2024-06-01"
                ,"feeType":("FactorFee", 0.01, 25, "Up")}
              ,{"status":"Current"}]
pool = {'assets':[receivable1,receivable4],'issuanceStat':{"IssuanceBalance":100000}}

dates={"collect": ["2024-02-29","2024-03-31"]
        ,"pay": ["2024-02-15"  
                      ,"2024-03-15"]  
        ,"stated":"2027-11-04"
        ,"poolFreq":"MonthFirst"
        ,"payFreq":["DayOfMonth",15]
        }

accounts={"collections":{"balance":50_000_000},
            "revolBuyAcc":{"balance":100_000}
            ,"purchaserAccount": {"balance":0}
            ,"sellerAccount": {"balance": 0}
          ,"cashReserve":{"balance":0}}
          
bonds={"A1":{"balance":150_000_000
                    ,"rate":0.15
                    ,"originBalance":200_000_000
                    ,"originRate":0.0858
                    ,"startDate":"2022-11-04"
                    ,"rateType":{"fix": 0.15, "dayCount":"DC_ACT_365A"}
                    ,"maturityDate":"2027-11-04"
                    ,"bondType":{"Lockout":"2023-11-04"}}
                    ,"B":{
                    "balance":1
                    ,"rate":0.0
                    ,"originBalance":1000
                    ,"originRate":0.0
                    ,"startDate":"2022-11-04"
                    ,"rateType":{"Fixed":0.00}
                    ,"bondType":{"Equity":None}
                    }
                    }

fees={"technology_fee":{"type":{"annualPctFee":[("bondBalance",),0.004]}}}

waterfall = {"Revolving": [
              ["payFee","collections",["technology_fee"]]
              ,["accrueAndPayInt","collections",["A1"]]
              ,["transfer" ,"collections","revolBuyAcc"]
              ,["buyAsset",["Current|Defaulted",1.0,0],"revolBuyAcc",None] #This line is the problematic one. If I remove it the server will run the deal
              ,["payIntResidual","collections","B"]]
  
  ,"endOfCollection":[["calcFee","technology_fee"] ,["calcInt","A1"]]

        ,"cleanUp":[["sellAsset", ["Current|Defaulted",1.0,0], "collections"] #I also comment out this line as well
                    ,["transfer" ,"cashReserve",'collections']
                    ,["transfer" ,"revolBuyAcc",'collections']
                    ,["accrueAndPayInt","collections",["A1"]]
                    ,["payPrin","collections",["A1"]]
                    ,["payIntResidual","collections","B"]]}
        
collects=[#["CollectedCash","collections"]
          ["CollectedPrincipal","collections"]
          ,["CollectedFeePaid","collections"]]

trigger=None
deal_data = {
          "name":"Deal"
          ,"dates":dates
          ,"pool":pool
          ,"accounts":accounts
          ,"fees":fees
          ,"bonds":bonds
          ,"waterfall":waterfall
          ,"collect":collects
          ,"trigger":trigger
          ,"status":"Revolving"
        } 

from absbox import mkDeal, API, EnginePath
deal=mkDeal(deal_data)


revol_asset = ["Invoice",
                  {
                    "start":"2021-01-01",
                    "originBalance":20000,
                    "originAdvance":18000,
                    "dueDate":"2021-04-01",
                    "feeType":("FixedRate",0.069)
                }
                  ,{
                    "status":"current"
                    }]
revolving=("revolving"
        ,["constant",revol_asset]
        ,("Pool",("Receivable",{"CDR":0.03},None,None),None,None))
      
#localAPI = API("https://absbox.org/api/dev",lang='english')
localAPI = API(EnginePath.LOCAL,lang='english', check=False)

r = localAPI.run(deal
              ,poolAssump = ("Pool",("Receivable",{"CDR":0.01},None,None),None,None)
              ,runAssump = [("inspect"
                                ,("MonthEnd",("status",("Amortizing")))
                                ,("MonthEnd",("accountBalance",))
                                ,("MonthEnd",("poolBalance",))
                                ,("MonthEnd",("bondBalance",))
                                )
                            ,revolving
                            ,("call",{"afterDate":"2025-11-04"})
                            #,("stop","2024-05-30")
                          ]
            ,read=True)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.26.4, server:0.27.8

Warning Message from server:
Bond A1 is not paid off
Bond B is not paid off

In [58]:
import pandas
pandas.set_option('display.max_rows', None)
pandas.set_option("styler.format.thousands", ",")

In [59]:
df = r['pool']['flow']
df

,Balance,AccuredFee,Principal,Fee,Default,Recovery,Loss,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,
2024-03-31,48321213.12,0,0.00,0.00,0.00,0,0.00,0,0,0,0.00,0,0.00
2024-04-01,48321213.12,0,0.00,0.00,0.00,0,0.00,0,0,0,0.00,0,0.00
2024-05-01,48321213.12,0,0.00,0.00,0.00,0,0.00,0,0,0,0.00,0,0.00
2024-06-01,48319213.12,0,1936.76,59.89,3.35,0,3.35,0,0,0,3.35,0,3.35
2024-07-01,2000.00,0,44646744.16,3308942.36,361526.60,0,361526.60,0,0,0,361529.95,0,361529.95
2024-08-01,40044637.53,0,1843.81,149.49,6.70,0,6.70,0,0,0,361536.65,0,361536.65
2024-09-01,40044637.53,0,0.00,0.00,0.00,0,0.00,0,0,0,361536.65,0,361536.65
2024-10-01,40044637.53,0,0.00,0.00,0.00,0,0.00,0,0,0,361536.65,0,361536.65
2024-11-01,0.00,0,37002603.66,2742405.63,299628.24,0,299628.24,0,0,0,661164.89,0,661164.89


In [34]:
df = r['pool']['flow']
df

,Balance,AccuredFee,Principal,Fee,Default,Recovery,Loss,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,
2024-03-31,48321213.12,0,0.00,0.00,0.00,0,0.00,0,0,0,0.00,0,0.00
2024-04-01,48321213.12,0,0.00,0.00,0.00,0,0.00,0,0,0,0.00,0,0.00
2024-05-01,48321213.12,0,0.00,0.00,0.00,0,0.00,0,0,0,0.00,0,0.00
2024-06-01,48319213.12,0,1936.76,59.89,3.35,0,3.35,0,0,0,3.35,0,3.35
2024-07-01,2000.00,0,44646744.16,3308942.36,361526.60,0,361526.60,0,0,0,361529.95,0,361529.95
2024-08-01,40044637.53,0,1843.81,149.49,6.70,0,6.70,0,0,0,361536.65,0,361536.65
2024-09-01,40044637.53,0,0.00,0.00,0.00,0,0.00,0,0,0,361536.65,0,361536.65
2024-10-01,40044637.53,0,0.00,0.00,0.00,0,0.00,0,0,0,361536.65,0,361536.65
2024-11-01,0.00,0,37002603.66,2742405.63,299628.24,0,299628.24,0,0,0,661164.89,0,661164.89


In [38]:
df['Default'].cumsum()

Date
2024-03-31          0.00
2024-04-01          0.00
2024-05-01          0.00
2024-06-01          3.35
2024-07-01     361529.95
2024-08-01     361536.65
2024-09-01     361536.65
2024-10-01     361536.65
2024-11-01     661164.89
2024-12-01     661164.89
2025-01-01     661164.89
2025-02-01     661164.89
2025-03-01     899051.54
2025-04-01     899051.54
2025-05-01     899051.54
2025-06-01     899051.54
2025-07-01    1077125.70
2025-08-01    1077125.70
2025-09-01    1077125.70
2025-10-01    1077125.70
2025-11-01    1194846.88
Name: Default, dtype: float64

In [51]:
(df['Default'].cumsum() - df['CumDefault'])<0.01

Date
2024-03-31    True
2024-04-01    True
2024-05-01    True
2024-06-01    True
2024-07-01    True
2024-08-01    True
2024-09-01    True
2024-10-01    True
2024-11-01    True
2024-12-01    True
2025-01-01    True
2025-02-01    True
2025-03-01    True
2025-04-01    True
2025-05-01    True
2025-06-01    True
2025-07-01    True
2025-08-01    True
2025-09-01    True
2025-10-01    True
2025-11-01    True
dtype: bool

In [52]:
(df['Loss'].cumsum() - df['CumLoss'])<0.01

Date
2024-03-31    True
2024-04-01    True
2024-05-01    True
2024-06-01    True
2024-07-01    True
2024-08-01    True
2024-09-01    True
2024-10-01    True
2024-11-01    True
2024-12-01    True
2025-01-01    True
2025-02-01    True
2025-03-01    True
2025-04-01    True
2025-05-01    True
2025-06-01    True
2025-07-01    True
2025-08-01    True
2025-09-01    True
2025-10-01    True
2025-11-01    True
dtype: bool